<a href="https://colab.research.google.com/github/gabrielcdo/Pythorch/blob/main/TripletMarginLossMNISTWithDoggos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip3 install pytorch-metric-learning
!pip3 install faiss-gpu
!pip3 install resnet_pytorch
!pip3 install split_folders

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
from pytorch_metric_learning import losses, miners, distances, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ### 
from torchvision import datasets
import torchvision.models as models
import torchvision
import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

import splitfolders
from resnet_pytorch import ResNet 

In [4]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ### 
def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print("Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(epoch, batch_idx, loss, mining_func.num_triplets))

In [5]:
### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

In [6]:
### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(test_embeddings, 
                                                train_embeddings,
                                                test_labels,
                                                train_labels,
                                                False)
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))

In [7]:
device = torch.device("cuda")
input_size = 224
transform_train = transforms.Compose([
                transforms.Resize((input_size, input_size)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(degrees=10),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
transform_test = transforms.Compose([
                transforms.Resize((input_size, input_size)),

                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

batch_size = 218

In [8]:
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
model1 = torchvision.models.resnet18(pretrained=True, progress=True)
model1.eval()
if torch.cuda.is_available():
    print("Cuda available")    
    model1.to('cuda')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth



Cuda available


In [18]:
dataset1 = datasets.ImageFolder('/content/drive/MyDrive/Models_training/doggos/test/ds-Flickr-dog-FD',transform=transform_train)

dataset2 = datasets.ImageFolder('/content/drive/MyDrive/Models_training/doggos/test/ds-Flickr-dog-cc',transform=transform_test)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=218, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, shuffle=True)


Dataset ImageFolder
    Number of datapoints: 223
    Root location: /content/drive/MyDrive/Models_training/doggos/test/ds-Flickr-dog-FD
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [30]:
input_folder = "/content/drive/MyDrive/Dataset Dogs/Images"
output = "/content/drive/MyDrive/Dataset Dogs" #where you want the split datasets saved. one will be created if none is set

splitfolders.ratio(input_folder, output="output", seed=42, ratio=(.8, .2)) # ratio of split are in order of train/val/test. You can change to whatever you want. For train/val sets only, you could do .75, .25 for example.



Copying files: 0 files [00:00, ? files/s]

Copying files: 1 files [00:00,  3.29 files/s]

Copying files: 2 files [00:00,  3.23 files/s]

Copying files: 3 files [00:00,  3.53 files/s]

Copying files: 4 files [00:01,  3.08 files/s]

Copying files: 5 files [00:01,  2.97 files/s]

Copying files: 6 files [00:02,  2.74 files/s]

Copying files: 7 files [00:02,  2.99 files/s]

Copying files: 8 files [00:02,  3.13 files/s]

Copying files: 9 files [00:02,  3.37 files/s]

Copying files: 10 files [00:03,  3.48 files/s]

Copying files: 11 files [00:03,  3.37 files/s]

Copying files: 12 files [00:06,  1.02 files/s]

Copying files: 13 files [00:06,  1.32 files/s]

Copying files: 14 files [00:06,  1.65 files/s]

Copying files: 15 files [00:06,  2.04 files/s]

Copying files: 16 files [00:06,  2.43 files/s]

Copying files: 17 files [00:07,  2.80 files/s]

Copying files: 18 files [00:07,  2.95 files/s]

Copying files: 19 files [00:07,  3.15 files/s]

Copying files: 20 files [00:08,  2.99 files/s]

Copy

KeyboardInterrupt: ignored

In [19]:
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
model1 = torchvision.models.resnet18(pretrained=True, progress=True)
model1.eval()
if torch.cuda.is_available():
    print("Cuda available")    
    model1.to('cuda')
optimizer = optim.Adam(model1.parameters(), lr=0.00003)


Cuda available


In [24]:
### pytorch-metric-learning stuff ###
distance = distances.LpDistance(power=2)
reducer = reducers.ThresholdReducer(low = 0)
loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance)
mining_func = miners.TripletMarginMiner(margin=0.2,type_of_triplets="easy", distance=distance)
accuracy_calculator = AccuracyCalculator(include = ("precision_at_1",), k = 1)

In [25]:
num_epochs = 100
### pytorch-metric-learning stuff ###
for epoch in range(1, num_epochs+1):
    train(model1, loss_func, mining_func, device, train_loader, optimizer, epoch)
    test(dataset1, dataset2, model1, accuracy_calculator)
    



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1 Iteration 0: Loss = 0.0, Number of mined triplets = 225127




 14%|█▍        | 1/7 [00:01<00:11,  1.96s/it]

 29%|██▊       | 2/7 [00:02<00:07,  1.41s/it]

100%|██████████| 7/7 [00:03<00:00,  2.25it/s]


  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:02<00:24,  2.23s/it]

 17%|█▋        | 2/12 [00:02<00:16,  1.67s/it]

 33%|███▎      | 4/12 [00:02<00:09,  1.19s/it]

 50%|█████     | 6/12 [00:02<00:05,  1.18it/s]

 75%|███████▌  | 9/12 [00:02<00:01,  1.66it/s]

100%|██████████| 12/12 [00:03<00:00,  3.13it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.660427807486631




  0%|          | 0/7 [00:00<?, ?it/s]

...

Computing accuracy
Test set accuracy (Precision@1) = 0.6711229946524063




  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 100 Iteration 0: Loss = 0.0, Number of mined triplets = 227987




 14%|█▍        | 1/7 [00:03<00:18,  3.13s/it]

 43%|████▎     | 3/7 [00:03<00:08,  2.21s/it]

100%|██████████| 7/7 [00:04<00:00,  1.65it/s]


  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:03<00:39,  3.57s/it]

 25%|██▌       | 3/12 [00:03<00:22,  2.51s/it]

 42%|████▏     | 5/12 [00:03<00:12,  1.78s/it]

 67%|██████▋   | 8/12 [00:03<00:05,  1.26s/it]

100%|██████████| 12/12 [00:04<00:00,  2.47it/s]

Computing accuracy
Test set accuracy (Precision@1) = 0.7005347593582887


In [ ]:
from graphviz import Digraph
from torch.autograd import Variable
import torch


def make_dot(var, params=None):
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style="filled", shape="box", align="left", fontsize="12", ranksep="0.1", height="0.2")
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return "(" + (", ").join(["%d" % v for v in size]) + ")"

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor="orange")
                dot.edge(str(id(var.grad_fn)), str(id(var)))
                var = var.grad_fn
            if hasattr(var, "variable"):
                u = var.variable
                name = param_map[id(u)] if params is not None else ""
                node_name = "%s\n %s" % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor="lightblue")
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, "next_functions"):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, "saved_tensors"):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)

    add_nodes(var)
    return dot


In [ ]:

import torchvision.models as models

inputs = torch.randn(1, 3, 224, 224)
resnet18 = models.resnet18()
y = resnet18(inputs)
# print(y)

g = make_dot(y)
print(g.view())